In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
#2022~2024 파일 전처리(2025 제외)
files = [f for f in glob.glob('../data/노선별*') if '2025' not in f]
print(files)

['../data\\노선별 항공사별 항공운송실적(2022년 전체).csv', '../data\\노선별 항공사별 항공운송실적(2023년 전체).csv', '../data\\노선별 항공사별 항공운송실적(2024년 전체).csv']


In [3]:
#2022~2024 데이터 컬럼명 통일, 병합
dfs = []

for f in files:
    df1 = pd.read_csv(f, encoding = 'cp949', dtype = str)

    if '경유지공항' in df1.columns:
        df1.rename(columns = {'경유지공항':'공항'}, inplace = True)
    
    if '항공화물(kg)' in df1.columns:
        df1.rename(columns = {'항공화물(kg)':'직화물(kg)'}, inplace = True)

    dfs.append(df1)

df = pd.concat(dfs, ignore_index = True)
df.head()

,년월,항공사(ICAO),항공사(IATA),항공사명,국제_국내,공항,도착_출발,정기_부정기,여객_화물,유임승객(명),무임승객(명),환승객(명),직화물(kg),환적화물(kg),우편물(kg),수하물(kg)
0,Jan-22,AAL,AA,아메리칸항공,국제,DFW,도착,정기,여객,2725,17,132,148646,0,21973,63814
1,Jan-22,AAL,AA,아메리칸항공,국제,DFW,도착,부정기,화물,0,0,0,394752,0,12163,0
2,Jan-22,AAL,AA,아메리칸항공,국제,DFW,출발,정기,여객,3703,26,107,229536,0,18442,86073
3,Jan-22,AAL,AA,아메리칸항공,국제,DFW,출발,부정기,화물,0,0,0,185997,0,5618,0
4,Jan-22,AAL,AA,아메리칸항공,국제,LAX,도착,부정기,화물,0,0,0,436208,0,53132,0


In [4]:
#결측치 확인
df.isnull().sum()

년월           0
항공사(ICAO)    0
항공사(IATA)    0
항공사명         0
국제_국내        0
공항           0
도착_출발        0
정기_부정기       0
여객_화물        0
유임승객(명)      0
무임승객(명)      0
환승객(명)       0
직화물(kg)      0
환적화물(kg)     0
우편물(kg)      0
수하물(kg)      0
dtype: int64

In [5]:
#년월 데이터 datetime 형태 변환, 년도/월 컬럼 만들기
import datetime

def 년월(x):
    x=str(x).strip()
    try:
        return pd.to_datetime(x, format='%b-%y')
    except:
        try:
            return pd.to_datetime(x, format='%Y-%m')
        except:
            return pd.NaT

df['년월_dt'] = df['년월'].apply(년월)
df["년도"] = df["년월_dt"].dt.year.astype("Int64")
df["월"] = df["년월_dt"].dt.month.astype("Int64") 
df.head()

,년월,항공사(ICAO),항공사(IATA),항공사명,국제_국내,공항,도착_출발,정기_부정기,여객_화물,유임승객(명),무임승객(명),환승객(명),직화물(kg),환적화물(kg),우편물(kg),수하물(kg),년월_dt,년도,월
0,Jan-22,AAL,AA,아메리칸항공,국제,DFW,도착,정기,여객,2725,17,132,148646,0,21973,63814,2022-01-01,2022,1
1,Jan-22,AAL,AA,아메리칸항공,국제,DFW,도착,부정기,화물,0,0,0,394752,0,12163,0,2022-01-01,2022,1
2,Jan-22,AAL,AA,아메리칸항공,국제,DFW,출발,정기,여객,3703,26,107,229536,0,18442,86073,2022-01-01,2022,1
3,Jan-22,AAL,AA,아메리칸항공,국제,DFW,출발,부정기,화물,0,0,0,185997,0,5618,0,2022-01-01,2022,1
4,Jan-22,AAL,AA,아메리칸항공,국제,LAX,도착,부정기,화물,0,0,0,436208,0,53132,0,2022-01-01,2022,1


In [6]:
#2022~2024 불필요 데이터 삭제
print(df.shape)
df = df.drop(['항공사(ICAO)','년월'], axis=1)
print(df.shape)

(37987, 19)
(37987, 17)


In [7]:
#2025 파일 전처리
df2 = pd.read_csv('../data/노선별 항공사별 항공운송실적(2025년 01~07).csv', encoding = 'cp949', dtype = str)
df2.head()

,년도,월,항공사(ICAO),항공사(IATA),항공사명,국제_국내,노선,국가,공항,도착_출발,정기_부정기,여객_화물,운항(편),유임승객(명),무임승객(명),환승객(명),직화물(kg),환적화물(kg),우편물(kg),수하물(kg)
0,2025,01,AAL,AA,아메리칸항공,국제,미주,미국,DFW,도착,정기,여객,29,5374,15,959,17112,0,10757,151120
1,2025,01,AAL,AA,아메리칸항공,국제,미주,미국,DFW,출발,정기,여객,29,4762,22,2938,226482,0,20853,160667
2,2025,01,AAR,OZ,아시아나항공,국내,한국,한국,CJU,도착,부정기,여객,1,232,0,0,0,0,0,1305
3,2025,01,AAR,OZ,아시아나항공,국제,일본,일본,AKJ,도착,정기,여객,17,2172,6,85,0,0,0,29890
4,2025,01,AAR,OZ,아시아나항공,국제,일본,일본,AKJ,출발,정기,여객,17,2436,5,100,0,0,0,29428


In [8]:
#2025년 데이터 불필요 컬럼 제거
print(df2.shape)
df2 = df2.drop(['항공사(ICAO)', '노선', '국가'], axis=1)
print(df2.shape)

(7227, 20)
(7227, 17)


In [9]:
#년도,월 컬럼 활용하여 시계열 데이터 만들기
df2['년월_dt'] = df2['년도'] + df2['월']
df2['년월_dt'] = pd.to_datetime(df2['년월_dt'], format = '%Y%m')
df2['년월_dt']

0      2025-01-01
1      2025-01-01
2      2025-01-01
3      2025-01-01
4      2025-01-01
          ...    
7222   2025-06-01
7223   2025-06-01
7224   2025-06-01
7225   2025-06-01
7226   2025-06-01
Name: 년월_dt, Length: 7227, dtype: datetime64[ns]

In [10]:
#2025년 데이터와 2022~2024년 데이터 병합
df = pd.concat([df, df2], ignore_index=True)
df.head()

,항공사(IATA),항공사명,국제_국내,공항,도착_출발,정기_부정기,여객_화물,유임승객(명),무임승객(명),환승객(명),직화물(kg),환적화물(kg),우편물(kg),수하물(kg),년월_dt,년도,월,운항(편)
0,AA,아메리칸항공,국제,DFW,도착,정기,여객,2725,17,132,148646,0,21973,63814,2022-01-01,2022,1,NaN
1,AA,아메리칸항공,국제,DFW,도착,부정기,화물,0,0,0,394752,0,12163,0,2022-01-01,2022,1,NaN
2,AA,아메리칸항공,국제,DFW,출발,정기,여객,3703,26,107,229536,0,18442,86073,2022-01-01,2022,1,NaN
3,AA,아메리칸항공,국제,DFW,출발,부정기,화물,0,0,0,185997,0,5618,0,2022-01-01,2022,1,NaN
4,AA,아메리칸항공,국제,LAX,도착,부정기,화물,0,0,0,436208,0,53132,0,2022-01-01,2022,1,NaN


In [11]:
#숫자형 컬럼 형태 변환
#변경 전
print(df.info())

cols = ['유임승객(명)', '무임승객(명)', '환승객(명)', '직화물(kg)', '환적화물(kg)','우편물(kg)','수하물(kg)', '운항(편)']
for col in cols:
    df[col] = pd.to_numeric(df[col], errors = 'coerce').astype('Int64')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45214 entries, 0 to 45213
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   항공사(IATA)  45214 non-null  object        
 1   항공사명       45214 non-null  object        
 2   국제_국내      45214 non-null  object        
 3   공항         45214 non-null  object        
 4   도착_출발      45214 non-null  object        
 5   정기_부정기     45214 non-null  object        
 6   여객_화물      45214 non-null  object        
 7   유임승객(명)    45214 non-null  object        
 8   무임승객(명)    45214 non-null  object        
 9   환승객(명)     45214 non-null  object        
 10  직화물(kg)    45214 non-null  object        
 11  환적화물(kg)   45214 non-null  object        
 12  우편물(kg)    45214 non-null  object        
 13  수하물(kg)    45214 non-null  object        
 14  년월_dt      45214 non-null  datetime64[ns]
 15  년도         45214 non-null  object        
 16  월          45214 non-null  object       

In [12]:
#변경 후
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45214 entries, 0 to 45213
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   항공사(IATA)  45214 non-null  object        
 1   항공사명       45214 non-null  object        
 2   국제_국내      45214 non-null  object        
 3   공항         45214 non-null  object        
 4   도착_출발      45214 non-null  object        
 5   정기_부정기     45214 non-null  object        
 6   여객_화물      45214 non-null  object        
 7   유임승객(명)    45214 non-null  Int64         
 8   무임승객(명)    45214 non-null  Int64         
 9   환승객(명)     45214 non-null  Int64         
 10  직화물(kg)    45214 non-null  Int64         
 11  환적화물(kg)   45214 non-null  Int64         
 12  우편물(kg)    45214 non-null  Int64         
 13  수하물(kg)    45214 non-null  Int64         
 14  년월_dt      45214 non-null  datetime64[ns]
 15  년도         45214 non-null  object        
 16  월          45214 non-null  object       

In [13]:
#총여객(명), 항공화물(kg), 평균탑승객(only 2025) 컬럼 생성
#총여객수 = 유임여객수+무임여객수+환승여객수
#항공화물 = 직화물+환적화물+우편물
#평균탑승객 = 총여객수/운항(편), 좌석 수 데이터 부재로 인해 탑승률의 proxy(대체지표)로 사용, 

df['총여객(명)'] = df['유임승객(명)'] + df['무임승객(명)'] + df['환승객(명)'] 
df['항공화물(kg)'] = df['직화물(kg)'] + df['환적화물(kg)'] + df['우편물(kg)']
df['평균탑승객(명/편)_proxy'] = round(df['총여객(명)'] / df['운항(편)'], 2)

df.head() 

,항공사(IATA),항공사명,국제_국내,공항,도착_출발,정기_부정기,여객_화물,유임승객(명),무임승객(명),환승객(명),...,환적화물(kg),우편물(kg),수하물(kg),년월_dt,년도,월,운항(편),총여객(명),항공화물(kg),평균탑승객(명/편)_proxy
0,AA,아메리칸항공,국제,DFW,도착,정기,여객,2725,17,132,...,0,21973,63814,2022-01-01,2022,1,<NA>,2874,170619,<NA>
1,AA,아메리칸항공,국제,DFW,도착,부정기,화물,0,0,0,...,0,12163,0,2022-01-01,2022,1,<NA>,0,406915,<NA>
2,AA,아메리칸항공,국제,DFW,출발,정기,여객,3703,26,107,...,0,18442,86073,2022-01-01,2022,1,<NA>,3836,247978,<NA>
3,AA,아메리칸항공,국제,DFW,출발,부정기,화물,0,0,0,...,0,5618,0,2022-01-01,2022,1,<NA>,0,191615,<NA>
4,AA,아메리칸항공,국제,LAX,도착,부정기,화물,0,0,0,...,0,53132,0,2022-01-01,2022,1,<NA>,0,489340,<NA>


In [14]:
#공항코드 데이터 불러오기
df1 = pd.read_csv('../data/국토교통부_세계공항_정보_20241231.csv', encoding='cp949', dtype = str)
df1 = df1.drop(['한글공항','공항코드2(ICAO)','영문국가명'], axis=1)
df1.head() 

,영문공항명,공항코드1(IATA),지역,한글국가명,영문도시명
0,A Coruna Airport,LCG,유럽,스페인,ACoruna
1,Aalborg Airport,AAL,유럽,덴마크,Aalborg
2,Aarhus Airport,AAR,유럽,덴마크,Aarhus
3,Aasiaat Airport,JEG,유럽,그린란드,Aasiaat
4,Aba Segud Airport,JIM,중동아프리카,이디오피아,Jimma


In [15]:
#매핑할 컬럼 공백 제거, 대문자화
df['공항'] = df['공항'].str.strip().str.upper()
df1['공항코드1(IATA)'] = df1['공항코드1(IATA)'].str.strip().str.upper()

In [16]:
#df_공항 컬럼, df1_공항코드(IATA) 매핑(영문공항명, 공항코드(IATA), 지역,  한글국가명, 영문도시명 활용)
print(df.shape)
merged = pd.merge(df, df1, left_on='공항', right_on = '공항코드1(IATA)', how = 'left')
print(merged.shape)

(45214, 21)
(45214, 26)


In [17]:
#매핑 데이터 결측치 확인
merged.isnull().sum()

항공사(IATA)               0
항공사명                    0
국제_국내                   0
공항                      0
도착_출발                   0
정기_부정기                  0
여객_화물                   0
유임승객(명)                 0
무임승객(명)                 0
환승객(명)                  0
직화물(kg)                 0
환적화물(kg)                0
우편물(kg)                 0
수하물(kg)                 0
년월_dt                   0
년도                      0
월                       0
운항(편)               37987
총여객(명)                  0
항공화물(kg)                0
평균탑승객(명/편)_proxy    37987
영문공항명                   8
공항코드1(IATA)             8
지역                      8
한글국가명                   8
영문도시명                   8
dtype: int64

In [18]:
#df['공항']컬럼에는 존재하지만, df1['공항코드1(IATA)']에는 존재하지 않는 데이터 확인
print(merged.loc[merged['공항코드1(IATA)'].isna(), '공항'])

#화물 데이터만 존재, 따로 저장/추후 보조자료로 활용
cargo_only = merged[merged['공항코드1(IATA)'].isna()].copy()
cargo_only.to_csv('../data/cargo_only.csv', encoding='cp949', index=False)

804      VDA
879      IAB
12735    AMB
20750    VBG
26536    QQS
34605    VBG
40599    QQS
44347    OSN
Name: 공항, dtype: object


In [19]:
#공항 코드 매칭 완료된 데이터만 조회
df = merged[merged['공항코드1(IATA)'].notna()].copy()
print(df.isnull().sum())

항공사(IATA)               0
항공사명                    0
국제_국내                   0
공항                      0
도착_출발                   0
정기_부정기                  0
여객_화물                   0
유임승객(명)                 0
무임승객(명)                 0
환승객(명)                  0
직화물(kg)                 0
환적화물(kg)                0
우편물(kg)                 0
수하물(kg)                 0
년월_dt                   0
년도                      0
월                       0
운항(편)               37981
총여객(명)                  0
항공화물(kg)                0
평균탑승객(명/편)_proxy    37981
영문공항명                   0
공항코드1(IATA)             0
지역                      0
한글국가명                   0
영문도시명                   0
dtype: int64


In [20]:
#공항,공항코드1(IATA) 값 일치하는지 확인
print((df['공항'] == df['공항코드1(IATA)']).all())

#공항코드1(IATA) 삭제
print(df.shape)
df = df.drop(['공항코드1(IATA)'], axis=1)
print(df.shape)

True
(45206, 26)
(45206, 25)


In [21]:
#여객용, 화물용 데이터 분리
passenger_cols = ['년월_dt', '년도', '월', '항공사(IATA)', '항공사명', '국제_국내', '정기_부정기', '도착_출발', '공항', '영문공항명', '지역', '한글국가명', '영문도시명', '여객_화물', '유임승객(명)', '무임승객(명)', '환승객(명)', '총여객(명)', '운항(편)', '평균탑승객(명/편)_proxy', '수하물(kg)']              
cargo_cols = ['년월_dt', '년도', '월', '항공사(IATA)', '항공사명', '국제_국내', '정기_부정기', '도착_출발', '공항', '영문공항명', '지역', '한글국가명', '영문도시명', '여객_화물', '직화물(kg)', '환적화물(kg)', '우편물(kg)', '항공화물(kg)', '운항(편)']

passenger_df = df[[p for p in passenger_cols if p in df.columns]]
cargo_df = df[[c for c in cargo_cols if c in df.columns]]

passenger_df.head()
cargo_df.tail()

,년월_dt,년도,월,항공사(IATA),항공사명,국제_국내,정기_부정기,도착_출발,공항,영문공항명,지역,한글국가명,영문도시명,여객_화물,직화물(kg),환적화물(kg),우편물(kg),항공화물(kg),운항(편)
45209,2025-06-01,2025,06,WS,웨스트젯 항공,국제,정기,출발,YYC,Calgary International Airport,북미캐나다,캐나다,Calgary,여객,351869,0,0,351869,26
45210,2025-06-01,2025,06,D7,에어아시아 엑스,국제,정기,도착,KUL,Kuala Lumpur International Airport,아시아태평양,말레이시아,Kuala Lumpur,여객,211995,0,7024,219019,34
45211,2025-06-01,2025,06,D7,에어아시아 엑스,국제,정기,출발,KUL,Kuala Lumpur International Airport,아시아태평양,말레이시아,Kuala Lumpur,여객,301140,0,0,301140,34
45212,2025-06-01,2025,06,D7,에어아시아 엑스,국제,부정기,도착,BSR,Basra International Airport,중동아프리카,이라크,Basra,여객,0,0,0,0,1
45213,2025-06-01,2025,06,D7,에어아시아 엑스,국제,부정기,출발,BSR,Basra International Airport,중동아프리카,이라크,Basra,여객,0,0,0,0,1


In [25]:
#여객용, 화물용 데이터 따로 저장
passenger_df.to_csv('../data/passenger_analysis.csv', encoding='cp949', index=False)
cargo_df.to_csv('../data/cargo_analysis.csv', encoding='cp949', index=False)